In [2]:
import pandas as pd
!conda install -c conda-forge pandas pgvector psycopg[binary] google-generativeai -y

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\neypi.NEY-DESKTOP\miniconda3\envs\seeding_data

  added / updated specs:
    - google-generativeai
    - pandas
    - pgvector
    - psycopg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiohttp-3.10.10            |  py310h38315fa_0         667 KB  conda-forge
    cachetools-5.5.0           |     pyhd8ed1ab_0          14 KB  conda-forge
    cryptography-43.0.3        |  py310h25a5809_0         1.2 MB  conda-forge
    frozenlist-1.5.0           |  py310ha8f682b_0          53 KB  conda-forge
    google-ai-generativelanguage-0.6.10|     pyhd8ed1ab_0         123 KB  conda-forge
    google-api-core-2.21.0     |     pyhd8ed1ab_0          87 KB  conda-forge
    google-api-python-client-2.149.0|     pyhff2d567_0         5.7 MB  conda-forge
    go

In [5]:
!pip install -U langchain-google-genai langchain langchain-postgres python-dotenv

In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from getpass import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("API_KEY")

In [5]:
import pandas as pd

[Baixe aqui os dados](https://arquivos.receitafederal.gov.br/cnpj/dados_abertos_cnpj/2024-10/Empresas0.zip)

[Metadata](https://www.gov.br/receitafederal/dados/cnpj-metadados.pdf)

In [ ]:
# download to data folder
!wget -P data https://arquivos.receitafederal.gov.br/cnpj/dados_abertos_cnpj/2024-10/Empresas0.zip

In [7]:
path = "data/K3241.K03200Y0.D41012.EMPRECSV"

In [11]:
from enum import Enum


class Porte(str, Enum):
    UNKNOWN = "00"
    MICRO = "01"
    PEQUENO = "03"
    DEMAIS = "05"

In [52]:
column_names = ["cnpj", "razao_social", "natureza_juridica", "qualificacao_responsavel", "capital_social", "porte", "ente_responsavel"]
column_types = {
    "cnpj": str,
    "razao_social": str,
    "natureza_juridica": str,
    "qualificacao_responsavel": str,
    "capital_social": str,
    "porte": str,
    "ente_responsavel": str
}

df = pd.read_csv(path, sep=";", encoding="latin1", header=None, names=column_names, dtype=column_types)

In [53]:
df

,cnpj,razao_social,natureza_juridica,qualificacao_responsavel,capital_social,porte,ente_responsavel
0,41273593,JULIO CESAR NUNES 39611300867,2135,50,"3000,00",01,NaN
1,41273594,OZINETE DELFINO CALDAS 41608224287,2135,50,"5000,00",01,NaN
2,41273595,GILVAN PEREIRA XAVIER 01363090380,2135,50,"3000,00",01,NaN
3,41273596,RODRIGO JOSE FERREIRA LOPES 05010247941,2135,50,"10000,00",01,NaN
4,41273597,PACHARRUS QUEIROZ DA COSTA E SILVA 03618384335,2135,50,"5000,00",01,NaN
...,...,...,...,...,...,...,...
19501127,98819550,ANTONIO ORIQUES CARDOSO,2135,50,"0,00",01,NaN
19501128,98819568,ELI ORIQUES CARDOSO,2135,50,"0,00",01,NaN
19501129,98819600,PAULINO LEMOS DA SILVA,2135,50,"0,00",05,NaN
19501130,98819832,JOAO FELTRIN,2135,50,"0,00",01,NaN


In [54]:
df['capital_social'] = df['capital_social'].str.replace(",", ".").astype(float)

In [55]:
df["porte"] = df["porte"].map(lambda x: Porte(x) if x in Porte._value2member_map_ else Porte.UNKNOWN)

In [56]:
df["porte"].value_counts()

porte
Porte.MICRO      14601345
Porte.DEMAIS      4331543
Porte.PEQUENO      554305
Porte.UNKNOWN       13939
Name: count, dtype: int64

In [65]:
df.where(pd.notnull(df), None, inplace=True)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19501132 entries, 0 to 19501131
Data columns (total 9 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   cnpj                      object 
 1   razao_social              object 
 2   natureza_juridica         object 
 3   qualificacao_responsavel  object 
 4   capital_social            float64
 5   porte                     object 
 6   ente_responsavel          object 
 7   document                  object 
 8   metadata                  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ GB


In [58]:
df.head()

,cnpj,razao_social,natureza_juridica,qualificacao_responsavel,capital_social,porte,ente_responsavel
0,41273593,JULIO CESAR NUNES 39611300867,2135,50,3000.0,Porte.MICRO,NaN
1,41273594,OZINETE DELFINO CALDAS 41608224287,2135,50,5000.0,Porte.MICRO,NaN
2,41273595,GILVAN PEREIRA XAVIER 01363090380,2135,50,3000.0,Porte.MICRO,NaN
3,41273596,RODRIGO JOSE FERREIRA LOPES 05010247941,2135,50,10000.0,Porte.MICRO,NaN
4,41273597,PACHARRUS QUEIROZ DA COSTA E SILVA 03618384335,2135,50,5000.0,Porte.MICRO,NaN


| Column Name                | Data Type      | Description                                                                                                  |
|----------------------------|----------------|--------------------------------------------------------------------------------------------------------------|
| `cnpj`                     | `str`          | The unique identifier for the company (CNPJ).                                                                |
| `razao_social`             | `str`          | The legal name of the company.                                                                               |
| `natureza_juridica`        | `str`          | The legal nature of the company, represented by a code in Annex V of Normative Instruction RFB No. 2119/2022 |
| `qualificacao_responsavel` | `str`          | The qualification of the responsible person, represented by a code.                                          |
| `capital_social`           | `float`        | The social capital of the company, represented as a decimal number.                                          |
| `porte`                    | `Porte` (Enum) | The size of the company, represented by an enum (`Porte.MICRO`, etc.).                                       |
| `ente_responsavel`         | `str`          | The responsible entity. Only for `natureza_juridica` of type 1xxx, the rest can be `null` as not applicable. |

### Enum `Porte`
| Enum Value     | Description                |
|----------------|----------------------------|
| `UNKNOWN`      | Unknown size               |
| `MICRO`        | Micro size company         |
| `PEQUENO`      | Small size company         |
| `DEMAIS`       | Other sizes                |

# Ingestion

In [59]:
df["document"] = df.apply(
    lambda row: f"Uma companhia nomeada como {row['razao_social']} com CNPJ {row['cnpj']} tem natureza jurídica com código {row['natureza_juridica']} com seu ente responsavel de {row['ente_responsavel']}, com a pessoa responsavel qualificada como {row['qualificacao_responsavel']}, capital social de {row['capital_social']} e possui porte de {row['porte']}",
    axis=1
)

In [68]:
df["document"] 

0           Uma companhia nomeada como JULIO CESAR NUNES 3...
1           Uma companhia nomeada como OZINETE DELFINO CAL...
2           Uma companhia nomeada como GILVAN PEREIRA XAVI...
3           Uma companhia nomeada como RODRIGO JOSE FERREI...
4           Uma companhia nomeada como PACHARRUS QUEIROZ D...
                                  ...                        
19501127    Uma companhia nomeada como ANTONIO ORIQUES CAR...
19501128    Uma companhia nomeada como ELI ORIQUES CARDOSO...
19501129    Uma companhia nomeada como PAULINO LEMOS DA SI...
19501130    Uma companhia nomeada como JOAO FELTRIN com CN...
19501131    Uma companhia nomeada como SOCIEDADE UNIAO DE ...
Name: document, Length: 19501132, dtype: object

In [69]:
df["metadata"] = df.apply(
    lambda row: {
        "cnpj": row["cnpj"],
        "razao_social": row["razao_social"],
        "natureza_juridica": row["natureza_juridica"],
        "qualificacao_responsavel": row["qualificacao_responsavel"],
        "capital_social": row["capital_social"],
        "porte": row["porte"],
        "ente_responsavel": row["ente_responsavel"]
    },
    axis=1
)

In [73]:
from sqlalchemy import create_engine, Column, Integer, String, JSON, ForeignKey, URL
from sqlalchemy.orm import sessionmaker, declarative_base
from pgvector.sqlalchemy import Vector
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Define the ORM base and model
Base = declarative_base()

class Collections(Base):
    __tablename__ = 'collections'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    description = Column(String, nullable=True)

class Embeddings(Base):
    __tablename__ = 'embeddings'
    id = Column(Integer, primary_key=True, autoincrement=True)
    collection_id = Column(Integer, ForeignKey(Collection.id))
    document = Column(String)
    cmetadata = Column(JSON)
    embedding = Column(Vector(768))  # AJUSTE O TAMANHO DO VETOR DE ACORDO COM O MODELO UTILIZADO (OLHAR A DOCUMENTAÇÃO DO MODELO)

In [74]:
connection_uri = URL.create(
    drivername="postgresql+psycopg", username=os.getenv("USERNAME"), password=os.getenv("PASSWORD"), host=os.getenv("HOSTNAME"), port=os.getenv("PORT"), database=os.getenv("DBNAME") # AJUSTE A CONEXÃO COM O BANCO DE DADOS CONFORME O SEU AMBIENTE
)

In [ ]:
# Connect to the PostgreSQL database
engine = create_engine(connection_uri)
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)

In [48]:
# Initialize the embeddings model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from sqlalchemy.exc import SQLAlchemyError

try:
    with Session() as session:
        # Insert the collection data
        collection = Collections(name="companies", description="Company data")
        session.add(collection)
        session.commit()

        # Prepare the list of Embeddings objects
        embeddings_list = []
        for index, row in df.iterrows():
            vector = embeddings.embed_query(row["document"])
            company_data = Embeddings(
                collection_id=collection.id,
                document=row["document"],
                cmetadata=row["metadata"],
                embedding=vector
            )
            embeddings_list.append(company_data)

        # Bulk insert all Embeddings objects
        session.bulk_save_objects(embeddings_list)
        session.commit()

except SQLAlchemyError as e:
    print(f"An error occurred: {e}")
    session.rollback()